In [0]:
#!hostname -I


In [2]:
!wget https://www.dropbox.com/s/4fm193mtun5nzl0/big_text_file.txt  # big_text_file.txt file of 4.00 MB that will be sent
#!wget https://www.dropbox.com/s/g8oq6mm2j1zhkv2/test_file.txt


--2018-05-06 20:31:55--  https://www.dropbox.com/s/4fm193mtun5nzl0/big_text_file.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/UKU08B7cxeWjQNJ1vC10hhRy5AsFLhMAzklCAg6gtq7dyZ8UcGuB5IddXTk0vabc/file [following]
--2018-05-06 20:31:56--  https://dl.dropboxusercontent.com/content_link/UKU08B7cxeWjQNJ1vC10hhRy5AsFLhMAzklCAg6gtq7dyZ8UcGuB5IddXTk0vabc/file
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.8.6, 2620:100:601f:6::a27d:906
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.8.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4194304 (4.0M) [text/plain]
Saving to: ‘big_text_file.txt’

big_text_file.txt   100%[===================>]   4.00M  12.8MB/s    in 0.3s    

2018-05-06 20:31

In [1]:
#!rm -r test_file.txt.1
!ls

datalab


In [0]:
import socket, pickle
receiverIP=socket.gethostbyname(socket.gethostname())
from socket import *
senderPort = 12000
receiverPort = 5200 

In [0]:
class Packet:
  def __init__(
  self,
  seq_num=None,
  checksum=None,
  data=None,
  type_ =1
  ):
    self.seq_num=seq_num
    self.checksum=checksum
    self.data=data
    self.type_=type_

class ACK:
  def __init__(
  self,
  seq_num=None,
  checksum=None,
  real=None
  ):
    self.seq_num=seq_num
    self.checksum=checksum
    self.real=real
  

In [0]:
import random

def decision(probability):
    return random.random() < probability

In [0]:
import csv

def file_to_pkt(file_name, sw, seq_num_range):
    packets = []
    f = open(file_name, 'rb')
    data = f.read(1024)
    seqNO = 0
    while data:
        if sw:
            packets.append(Packet(seqNO % 2, 0, data))

        else:
            packets.append(Packet(seqNO % seq_num_range, 0, data))
        data = f.read(1024)
        
        seqNO += 1
        
    packets[len(packets) - 1].type_ = 0
    return packets
  

def pkt_to_file(file_name, packets):
    f = open(file_name, 'wb')
    for i in range(len(packets)):
        f.write(packets[i].data)
    f.close()
    
def get_stats(file_name,stats_dict):
  with open(file_name, "w") as f:
    writer = csv.writer(f, delimiter="\t", lineterminator="\r\n")
    writer.writerows(stats_dict.items())
    


In [0]:
# import sys
# packets=[]
# packets = file_to_pkt(file_name='test_file.txt', sw=True)
# # print(sys.getsizeof(packets))
# print(len(packets))

## Stop and Wait

In [9]:
#@title Parameters Tuning { form-width: "45%"}
sent_file_name = "big_text_file.txt" #@param {type:"string"}
recv_file_name = "down_test_file_1.txt" #@param {type:"string"}
prob = 0.9 #@param {type:"slider", min:0, max:1, step:0.01}


import os
import time
from google.colab import files

start_time = time.time()


def recv_child():
    stats_ = {}
    receiverSocket = socket(AF_INET, SOCK_DGRAM)
    receiverSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    receiverSocket.bind(('', receiverPort))
    
    print ('--The receiver is ready to receive--\n')

    packets = []
    actual_seq_num = 0
    expected_seq_num = 0
    while True:

        (message, senderAddress) = receiverSocket.recvfrom(4096)
        message = pickle.loads(message)
        t6 = time.time()
        
        if expected_seq_num == message.seq_num:
            packets.append(message)
            if decision(prob):
                receiverSocket.sendto(pickle.dumps(ACK(expected_seq_num,
                        0, actual_seq_num)), senderAddress)
                t6 = time.time()
                stats_['{0:.6f}'.format(t6 - start_time)] = \
                    'receiver sent ack for packet ' \
                    + str(expected_seq_num) + ' actual: ' \
                    + str(actual_seq_num)
                actual_seq_num += 1
                expected_seq_num = actual_seq_num % 2

                if message.type_ == 0:
                    time.sleep(0.1)
                    
                    print ('--Whole file received successfully. File is ready to download--\n')

                    get_stats('receiver_stats.txt', stats_)

                    pkt_to_file(recv_file_name, packets)
            else:

                stats_['{0:.6f}'.format(t6 - start_time)] = \
                    'receiver dropped ack for packet ' \
                    + str(expected_seq_num) + ' actual: ' \
                    + str(actual_seq_num)
                actual_seq_num += 1
                expected_seq_num = actual_seq_num % 2
        else:

            # duplicate

            receiverSocket.sendto(pickle.dumps(ACK(message.seq_num, 0,
                                  actual_seq_num - 1)), senderAddress)

            stats_['{0:.6f}'.format(t6 - start_time)] = \
                '--duplicate-- already received ' \
                + str(message.seq_num) + ' actual: ' \
                + str(actual_seq_num - 1)



def send_parent():
  
    newpid = os.fork()
    if newpid == 0:
        recv_child()
    else:
        stats_ = {}
        time.sleep(3)
        pids = (os.getpid(), newpid)
        packets = []

        packets = file_to_pkt(file_name=sent_file_name, sw=True,  seq_num_range= 0)

        senderSocket = socket(AF_INET, SOCK_DGRAM)
        senderSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
        senderSocket.bind(('', senderPort))
        message = packets

        for i in range(len(packets)):
            if decision(prob):
                senderSocket.sendto(pickle.dumps(message[i]),
                                    (receiverIP, receiverPort))
                t1 = time.time()
                stats_['{0:.6f}'.format(t1 - start_time)] = 'sent packet ' \
                    + str(i % 2) + ' actual: ' + str(i)
            else:
                t1 = time.time()
                stats_['{0:.6f}'.format(t1 - start_time)] = 'sent packet ' \
                    + str(i % 2) + ' actual: ' + str(i)

            while True:

                modifiedMessage = pot_ack = None
                try:
                    senderSocket.settimeout(0.01)
                    (modifiedMessage, receiverAddress) = \
                        senderSocket.recvfrom(4096)
                    t5 = time.time()
                    pot_ack = pickle.loads(modifiedMessage)
                    if isinstance(pot_ack, ACK):
                        if pot_ack.seq_num == i % 2:
                            stats_['{0:.6f}'.format(t5 - start_time)] = \
                                'sender received ack for packet ' \
                                + str(pot_ack.seq_num) + ' actual: ' \
                                + str(pot_ack.real)
                            modifiedMessage = None

                            if i == len(packets) - 1:
                                get_stats('sender_stats.txt', stats_)
                                print ('--Whole file sent successfully--\n')

                        break
                except:
                    senderSocket.settimeout(None)
                    t8 = time.time()
                    stats_['{0:.6f}'.format(t8 - start_time)] = \
                        '--timeout-- no ack received for packet ' \
                        + str(i % 2) + ' actual: ' + str(i)

                    if decision(prob):
                        senderSocket.sendto(pickle.dumps(message[i]),
                                (receiverIP, receiverPort))
                        t7 = time.time()
                        stats_['{0:.6f}'.format(t7 - start_time)] = \
                            '--again-- sent packet ' + str(i % 2) \
                            + ' actual: ' + str(i)
                    else:
                        t7 = time.time()
                        stats_['{0:.6f}'.format(t7 - start_time)] = \
                            '--again-- dropped packet ' + str(i % 2) \
                            + ' actual: ' + str(i)

    senderSocket.close()


send_parent()


--The receiver is ready to receive--

--Whole file sent successfully--

--Whole file received successfully. File is ready to download--



In [0]:
#!ps -fA | grep python
from google.colab import files
files.download(recv_file_name)
files.download('receiver_stats.txt')
files.download('sender_stats.txt')

In [0]:
from google.colab import files
# !ls
# files.download('co_test_file.txt')


## Selective Repeat

In [0]:
#@title Parameters Tuning { form-width: "45%"}
sent_file_name = "big_text_file.txt" #@param {type:"string"}
recv_file_name = "down_test_file_1.txt" #@param {type:"string"}
prob = 0.9 #@param {type:"slider", min:0, max:1, step:0.01}
window_size = 5 #@param {type:"integer"}
seq_num_range = 10 #@param {type:"integer"}


import os
import time
from google.colab import files

start_time = time.time()


def recv_child():
    stats_ = {}
    receiverSocket = socket(AF_INET, SOCK_DGRAM)
    receiverSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    receiverSocket.bind(('', receiverPort))
    
    print ('--The receiver is ready to receive--\n')

    packets = []
    actual_seq_num = 0
    expected_seq_num = 0
    while True:

 


def send_parent():
  
    newpid = os.fork()
    if newpid == 0:
        recv_child()
    else:
        stats_ = {}
        time.sleep(3)
        packets = file_to_pkt(file_name=sent_file_name, sw=False, seq_num_range)
        
        
        
    senderSocket.close()


send_parent()


In [0]:
#!ps -fA | grep python
files.download(recv_file_name)
files.download('receiver_stats.txt')
files.download('sender_stats.txt')